In [20]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [5]:
df_large  = pd.read_excel("Business_Dataset.xlsx")
df_large.to_pickle('my_dataframe.pkl')

In [6]:
df = pd.read_pickle('my_dataframe.pkl')

In [24]:
df = df_large
print(len(df))
df.head(5)

245955


,tbl,Year,quarter,citymarketid_1,citymarketid_2,city1,city2,airportid_1,airportid_2,airport_1,...,fare,carrier_lg,large_ms,fare_lg,carrier_low,lf_ms,fare_low,Geocoded_City1,Geocoded_City2,tbl1apk
0,Table1a,2021,3,30135,33195,"Allentown/Bethlehem/Easton, PA","Tampa, FL (Metropolitan Area)",10135,14112,ABE,...,81.43,G4,1.0000,81.43,G4,1.0000,81.43,NaN,NaN,202131013514112ABEPIE
1,Table1a,2021,3,30135,33195,"Allentown/Bethlehem/Easton, PA","Tampa, FL (Metropolitan Area)",10135,15304,ABE,...,208.93,DL,0.4659,219.98,UA,0.1193,154.11,NaN,NaN,202131013515304ABETPA
2,Table1a,2021,3,30140,30194,"Albuquerque, NM","Dallas/Fort Worth, TX",10140,11259,ABQ,...,184.56,WN,0.9968,184.44,WN,0.9968,184.44,NaN,NaN,202131014011259ABQDAL
3,Table1a,2021,3,30140,30194,"Albuquerque, NM","Dallas/Fort Worth, TX",10140,11298,ABQ,...,182.64,AA,0.9774,183.09,AA,0.9774,183.09,NaN,NaN,202131014011298ABQDFW
4,Table1a,2021,3,30140,30466,"Albuquerque, NM","Phoenix, AZ",10140,14107,ABQ,...,177.11,WN,0.6061,184.49,AA,0.3939,165.77,NaN,NaN,202131014014107ABQPHX


In [9]:
df = df.drop('tbl', axis = 1)

In [25]:
df.isnull().sum()

tbl                   0
Year                  0
quarter               0
citymarketid_1        0
citymarketid_2        0
city1                 0
city2                 0
airportid_1           0
airportid_2           0
airport_1             0
airport_2             0
nsmiles               0
passengers            0
fare                  0
carrier_lg         1540
large_ms           1540
fare_lg            1540
carrier_low        1612
lf_ms              1612
fare_low           1612
Geocoded_City1    39206
Geocoded_City2    39206
tbl1apk               0
dtype: int64

In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapi")

def geocode_city(city):
    try:
        location = geolocator.geocode(city)
        return (location.latitude, location.longitude) if location else None
    except:
        return None

df['Geocoded_City1'] = df['Geocoded_City1'].fillna(df['city1'].apply(geocode_city))
df['Geocoded_City2'] = df['Geocoded_City2'].fillna(df['city2'].apply(geocode_city))

In [26]:
#Finding cardinatiy for data 
cardinality ={}
for col in df.columns:
    cardinality[col] = df[col].nunique()
cardinality

{'tbl': 2,
 'Year': 31,
 'quarter': 4,
 'citymarketid_1': 141,
 'citymarketid_2': 128,
 'city1': 141,
 'city2': 128,
 'airportid_1': 188,
 'airportid_2': 173,
 'airport_1': 187,
 'airport_2': 173,
 'nsmiles': 1155,
 'passengers': 3883,
 'fare': 36323,
 'carrier_lg': 67,
 'large_ms': 7367,
 'fare_lg': 37508,
 'carrier_low': 71,
 'lf_ms': 9687,
 'fare_low': 32283,
 'Geocoded_City1': 238,
 'Geocoded_City2': 207,
 'tbl1apk': 245955}

In [12]:
value_counts = {}

for col in df.columns:
    for value in df[col]:
        if value in value_counts:
            value_counts[value] += 1
        else:
            value_counts[value] = 1

value_counts_df = pd.DataFrame(list(value_counts.items()), columns=['Value', 'Count'])

value_counts_df

,Value,Count
0,2021,8186
1,2022,8101
2,2010,8119
3,1998,8924
4,2009,8117
...,...,...
307873,202411541212953TYSLGA,1
307874,202411541211697TYSFLL,1
307875,202411541213303TYSMIA,1
307876,202411541214112TYSPIE,1


In [17]:
df = df['Year'] - df['Year'].min()


0    28
1    28
2    28
3    28
4    28
Name: Year, dtype: int64

In [18]:
df = pd.get_dummies(df_large, columns=['quarter'])
df.head(1)

,tbl,Year,citymarketid_1,citymarketid_2,city1,city2,airportid_1,airportid_2,airport_1,airport_2,...,carrier_low,lf_ms,fare_low,Geocoded_City1,Geocoded_City2,tbl1apk,quarter_1,quarter_2,quarter_3,quarter_4
0,Table1a,2021,30135,33195,"Allentown/Bethlehem/Easton, PA","Tampa, FL (Metropolitan Area)",10135,14112,ABE,PIE,...,G4,1.0,81.43,NaN,NaN,202131013514112ABEPIE,False,False,True,False


In [16]:
#New dataframe where airportid_1 and 
# airportid_2 are replaced by their target-encoded versions based on fare_low.

airportid_1_encoded = df.groupby('airportid_1')['fare_low'].mean().to_dict()
df['airportid_1_encoded'] = df['airportid_1'].map(airportid_1_encoded)
airportid_2_encoded = df.groupby('airportid_2')['fare_low'].mean().to_dict()
df['airportid_2_encoded'] = df['airportid_2'].map(airportid_2_encoded)
df.drop(['airportid_1', 'airportid_2'], axis=1, inplace=True)
df.head()

,tbl,Year,citymarketid_1,citymarketid_2,city1,city2,airport_1,airport_2,nsmiles,passengers,...,fare_low,Geocoded_City1,Geocoded_City2,tbl1apk,quarter_1,quarter_2,quarter_3,quarter_4,airportid_1_encoded,airportid_2_encoded
0,Table1a,2021,30135,33195,"Allentown/Bethlehem/Easton, PA","Tampa, FL (Metropolitan Area)",ABE,PIE,970,180,...,81.43,NaN,NaN,202131013514112ABEPIE,False,False,True,False,191.835909,129.117099
1,Table1a,2021,30135,33195,"Allentown/Bethlehem/Easton, PA","Tampa, FL (Metropolitan Area)",ABE,TPA,970,19,...,154.11,NaN,NaN,202131013515304ABETPA,False,False,True,False,191.835909,156.790121
2,Table1a,2021,30140,30194,"Albuquerque, NM","Dallas/Fort Worth, TX",ABQ,DAL,580,204,...,184.44,NaN,NaN,202131014011259ABQDAL,False,False,True,False,188.713407,182.397671
3,Table1a,2021,30140,30194,"Albuquerque, NM","Dallas/Fort Worth, TX",ABQ,DFW,580,264,...,183.09,NaN,NaN,202131014011298ABQDFW,False,False,True,False,188.713407,186.393335
4,Table1a,2021,30140,30466,"Albuquerque, NM","Phoenix, AZ",ABQ,PHX,328,398,...,165.77,NaN,NaN,202131014014107ABQPHX,False,False,True,False,188.713407,185.221374
